In [2]:
%load_ext autoreload
%autoreload 2
import time
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)
# start = "2010-01-21T00:30:00"
# end = "2010-01-21T01:30:00"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/dev/gridrad_{event_start.replace('-', '')}"
event_start

'2010-01-20'

In [3]:
# if output_parent.exists():
# shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create and save the dataset options
times_dict = {"start": start, "end": end}
gridrad_dict = {"event_start": event_start}
gridrad_options = data.option.GridRadSevereOptions(**times_dict, **gridrad_dict)
era5_dict = {"latitude_range": [27, 39], "longitude_range": [-102, -89]}
era5_pl_options = data.option.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.option.ERA5Options(**times_dict, **era5_dict)
data_options = data.option.DataOptions(
    datasets=[gridrad_options, era5_pl_options, era5_sl_options]
)
data_options.to_yaml(options_directory / "data.yml")
gridrad_options = data_options.dataset_by_name("gridrad")

# Create and save the grid_options dictionary 
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
# track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
# track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
# visualize_options = {
#     obj: visualize.option.runtime_options(obj, save=True, style="presentation")
#     for obj in ["mcs"]
# }
visualize_options = None

2024-11-01 20:54:55,536 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-01 20:54:55,538 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:54:55,539 - thor.data.option - INFO - Assigning default era5 pressure-levels options name and fields.
2024-11-01 20:54:55,540 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-01 20:54:55,542 - thor.data.option - INFO - Assigning default era5 single-levels options name and fields.
2024-11-01 20:54:55,543 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-01 20:54:55,575 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.


In [5]:
%load_ext memory_profiler

In [6]:
times = data.utils.generate_times(data_options.dataset_by_name("gridrad"))
args = [times, data_options.model_copy(), grid_options.copy()]
args += [track_options.model_copy(), visualize_options]
track.simultaneous_track(*args, output_directory=output_parent)

2024-11-01 20:15:13,425 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100120.
2024-11-01 20:15:13,427 - thor.track - INFO - Beginning simultaneous tracking.


2024-11-01 20:15:13,550 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-11-01 20:15:13,551 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-11-01 20:15:13,552 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-11-01 20:15:15,565 - thor.track - INFO - Processing hierarchy level 0.
2024-11-01 20:15:15,566 - thor.track - INFO - Tracking convective.
2024-11-01 20:15:27,621 - thor.track - INFO - Tracking middle.
2024-11-01 20:15:27,700 - thor.track - INFO - Tracking anvil.
2024-11-01 20:15:27,763 - thor.track - INFO - Processing hierarchy level 1.
2024-11-01 20:15:27,764 - thor.track - INFO - Tracking mcs.
2024-11-01 20:15:28,004 - thor.match.match - INFO - Matching mcs objects.
2024-11-01 20:15:28,010 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-11-01 20:15:28,075 - thor.track - INFO - Processing 2010-01-20T18:10:00.
2024-11-01 20:15:28,078 - thor.data.gridrad 

KeyboardInterrupt: 

In [4]:
# num_processes = int(0.75 * os.cpu_count())
# num_processes = os.cpu_count()
num_processes = 4
kwargs = {"initializer": parallel.initialize_process, "processes": num_processes}
with log.logging_listener(), multiprocessing.get_context("spawn").Pool(**kwargs) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        time.sleep(1)
        args = [i, time_interval, data_options.model_copy(), grid_options.copy()]
        args += [track_options.model_copy(), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-11-01 20:47:28,306 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100120/interval_0.
2024-11-01 20:47:28,307 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-01 20:47:28,357 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100120/interval_1.
2024-11-01 20:47:28,358 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-01 20:47:28,405 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100120/interval_2.
2024-11-01 20:47:28,406 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-01 20:47:28,419 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-11-01 20:47:28,420 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-11-01 20:47:28,420 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-11-01 20:47:28,484 - thor.tr

In [5]:
parallel.stitch_run(output_parent, intervals, cleanup=False)

2024-11-01 20:51:44,913 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-11-01 20:51:44,941 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:51:53,701 - thor.parallel - INFO - Stitching record files.
2024-11-01 20:51:53,787 - thor.parallel - INFO - Stitching attribute files.
2024-11-01 20:51:56,677 - thor.parallel - INFO - Stitching mask files.


In [6]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent, profile_dataset=None)
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent)

2024-11-01 20:52:04,602 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:52:04,771 - thor.data.option - INFO - Assigning default era5 pressure levels.


convective_contained  anvil_contained  \
time                universal_id                                          
2010-01-20 18:00:00 1                             True            False   
                    2                             True            False   
2010-01-20 18:10:00 1                             True            False   
                    2                            False            False   
2010-01-20 18:20:00 1                             True            False   
...                                                ...              ...   
2010-01-21 03:10:00 15                            True             True   
                    31                            True             True   
                    36                           False            False   
2010-01-21 03:20:00 15                            True             True   
                    31                            True             True   

                                  velocity  area  offset  major_axis  \
time                universal_id                                       
2010-01-20 18:00:00 1                 True  True    True        True   
                    2                 True  True    True        True   
2010-01-20 18:10:00 1                 True  True    True        True   
                    2                 True  True   False        True   
2010-01-20 18:20:00 1                 True  True    True        True   
...                                    ...   ...     ...         ...   
2010-01-21 03:10:00 15                True  True    True        True   
                    31                True  True    True        True   
                    36                True  True   False        True   
2010-01-21 03:20:00 15                True  True    True        True   
                    31                True  True    True        True   

                                  axis_ratio  duration  parents  
time                universal_id                                 
2010-01-20 18:00:00 1                   True      True    False  
                    2                  False     False    False  
2010-01-20 18:10:00 1                   True      True    False  
                    2                  False     False    False  
2010-01-20 18:20:00 1                  False      True    False  
...                                      ...       ...      ...  
2010-01-21 03:10:00 15                  True      True     True  
                    31                 False      True    False  
                    36                  True      True     True  
2010-01-21 03:20:00 15                  True      True     True  
                    31                 False      True    False  

[273 rows x 9 columns]

In [4]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
figure_options = visualize.option.horizontal_attribute_options(
    figure_name, style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64(start)

end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

2024-11-01 20:54:59,072 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:54:59,443 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:00:00.000000000.
2024-11-01 20:55:01,056 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:55:02,068 - thor.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:00:00.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-11-01 20:55:15,895 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:10:00.000000000.
2024-11-01 20:55:16,184 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:20:00.000000000.
2024-11-01 20:55:16,745 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:30:00.000000000.
2024-11-01 20:55:17,508 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:40:00.000000000.
2024-11-01 20:55:23,163 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:55:23,363 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:55:24,477 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:55:24,769 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 20:55:26,212 - thor.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:10:00.000000000.
2024-11-01 20:55:26,501 - thor.visualize.a